In [1]:
import cv2
import time
import matplotlib.pyplot as plt
from deepface import DeepFace
import torch
import torch.nn as nn
import torch.nn.functional as F

2025-08-29 17:52:58.647890: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 17:52:58.658456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756489978.668084     624 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756489978.670861     624 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756489978.678244     624 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

25-08-29 17:52:59 - Directory /root/.deepface has been created
25-08-29 17:52:59 - Directory /root/.deepface/weights has been created


In [2]:
import numpy as np


In [18]:
cap = cv2.VideoCapture(0)
frame_count = 0
detected_faces = []
faces = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    try:
        if frame_count % 3 == 0:
            faces = DeepFace.extract_faces(frame, detector_backend='mtcnn', enforce_detection=False)
            detected_faces.append(faces)
    except:
        faces = []

    # Draw rectangles for each detected face
    for face_data in faces:
        facial_area = face_data["facial_area"]
        x, y, w, h = facial_area["x"], facial_area["y"], facial_area["w"], facial_area["h"]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Osoba", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, (0, 255, 0), 2, cv2.LINE_AA)
    
    cv2.imshow("Webcam Face Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# detected_faces = []
cap.release()
cv2.destroyAllWindows()

In [20]:
for i, faces in enumerate(detected_faces):
    cropped_face = faces[0]["face"]
    cropped_face_uint8 = (cropped_face * 255).astype(np.uint8)
    cv2.imwrite(f"data/test/non_target_person/cropped_face_mateusz{i}.jpg", cv2.cvtColor(cropped_face_uint8, cv2.COLOR_RGB2BGR))

In [21]:
len(detected_faces)

150

In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace

In [1]:
input_dir = "data/random_people"      
output_dir = "data/train/non_target_person" 
os.makedirs(output_dir, exist_ok=True)
MAX_DIM = 1024  # maximum width or height to prevent memory issues

for fname in os.listdir(input_dir):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(input_dir, fname)
    img = cv2.imread(img_path)

    if img is None:
        print(f"Could not read {img_path}")
        continue

    h, w = img.shape[:2]
    scale = MAX_DIM / max(h, w) if max(h, w) > MAX_DIM else 1
    if scale < 1:
        img = cv2.resize(img, (int(w*scale), int(h*scale)))

    try:
        faces = DeepFace.extract_faces(img, detector_backend="mtcnn", enforce_detection=False)
    except Exception as e:
        print(f"Error extracting from {fname}: {e}")
        continue

    if len(faces) == 0:
        print(f"No face found in {fname}")
        continue

    cropped_face = faces[0]["face"]
    cropped_face_uint8 = (cropped_face * 255).astype(np.uint8)

    save_path = os.path.join(output_dir, f"target_{fname}")
    cv2.imwrite(save_path, cv2.cvtColor(cropped_face_uint8, cv2.COLOR_RGB2BGR))
    print(f"Saved {save_path}")

NameError: name 'os' is not defined

In [3]:
import os

In [2]:
# os.listdir("./data/train/non_target_person")

In [20]:
import os
import random
import shutil

src_dir = "./data/train/non_target_person"
dst_dir = "./data/val/non_target_person"

os.makedirs(dst_dir, exist_ok=True)

files = [f for f in os.listdir(src_dir) if f.startswith("cropped_") and os.path.isfile(os.path.join(src_dir, f))]

random.shuffle(files)
num_to_move = int(0.2 * len(files))
files_to_move = files[:num_to_move]

for f in files_to_move:
    shutil.move(os.path.join(src_dir, f), os.path.join(dst_dir, f))

print(f"Moved {len(files_to_move)} files from {src_dir} to {dst_dir}")


Moved 81 files from ./data/train/non_target_person to ./data/val/non_target_person


In [22]:
import os
import random
import shutil

def move_random_files(src_dir, dst_dir, sample_size):
    os.makedirs(dst_dir, exist_ok=True)
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]

    if sample_size > len(files):
        raise ValueError(f"Sample size {sample_size} is larger than number of files {len(files)}")

    sampled_files = random.sample(files, sample_size)

    for f in sampled_files:
        shutil.move(os.path.join(src_dir, f), os.path.join(dst_dir, f))

    print(f"Moved {len(sampled_files)} files to {dst_dir}")

src = r"data/train/target_person"
dst = r"data/val/target_person"
n = 140  
move_random_files(src, dst, n)


Moved 140 files to data/val/target_person
